In [1]:
import pandas as pd
import altair as alt

from statistics import median, mean

In [22]:
gs_exomiser = pd.read_csv("GS_filtering_defaults_combos_exomiser_default12.13.24.tsv", sep='\t')
es_exomiser = pd.read_csv("ES_filtering_defaults_combos_exomiser_default12.14.24.tsv", sep='\t')
gs_genomiser=pd.read_csv('GS_genomiser_filtering_defaults_combos_default_genomiser12.14.24.tsv', sep='\t')
table = pd.concat([gs_exomiser, es_exomiser, gs_genomiser])

table['Filtering'] =  table['GQ'] + '_' + table['VAF'].astype(str) 

table.to_csv('all_cohorts_all_data.tsv', sep='\t', index=None)

In [23]:
len(gs_exomiser[['ID', 'Diagnostic_Variant']].drop_duplicates())+len(es_exomiser[['ID', 'Diagnostic_Variant']].drop_duplicates())+len(gs_genomiser[['ID', 'Diagnostic_Variant']].drop_duplicates())


509

In [24]:
GQs = [10, 20, 30, 40, 50, 60]
VAFs = [[.10,.90], [.15,.85], [.20,.80], [.25,.75]]
data=[]
lost_dict={}
##OG
subset = table[(table['GQ']=='OG') & (table['VAF']=='OG')]
subset=subset.fillna(0)
med_rank=median(subset[subset['Rank']!=0]['Rank'])
mean_rank = mean(subset[subset['Rank']!=0]['Rank'])
lost = len(subset[subset['Rank']==0]) - 35
data.append(['OG', 'OG',med_rank, mean_rank,lost])
lost_dict['OG_OG'] = lost/(509-35) * 100

for min_GQ in GQs:
    for VAF_range in VAFs:
        subset = table[(table['GQ']==str(min_GQ)) & (table['VAF']==str(VAF_range))]
        subset=subset.fillna(0)
        med_rank=median(subset[subset['Rank']!=0]['Rank'])
        mean_rank = mean(subset[subset['Rank']!=0]['Rank'])

        lost = len(subset[subset['Rank']==0]) - 35
        print(med_rank, lost)
        data.append([str(min_GQ), VAF_range, med_rank,mean_rank, lost])
        lost_dict[str(min_GQ) + '_' + str(VAF_range)] = lost/(509-35) * 100
        


3.5 2
3.0 2
2.0 4
2.0 5
3.5 2
3.0 2
2.0 4
2.0 5
3.0 3
2.0 3
2.0 5
2.0 6
3.0 4
2.0 4
2.0 6
2.0 7
3.0 4
2.0 4
2.0 6
2.0 7
3.0 4
2.0 4
2.0 6
2.0 7


In [25]:
lost_row = []
for i, row in table.iterrows():
    lost = lost_dict[row['Filtering']]
    lost_row.append(lost)
table['Percent_Lost'] = lost_row
table


,ID,Diagnostic_Gene,Diagnostic_Variant,Rank,Class,GQ,VAF,Alt,Filtering,Percent_Lost
0,UDN004009,CTSA,20-45892776-GA-G,2.0,OG,OG,OG,OG,OG_OG,0.000000
1,UDN004009,CTSA,20-45892776-GA-G,2.0,Filtered,10,"[0.1, 0.9]",no*,"10_[0.1, 0.9]",0.421941
2,UDN004009,CTSA,20-45892776-GA-G,2.0,Filtered,10,"[0.15, 0.85]",no*,"10_[0.15, 0.85]",0.421941
3,UDN004009,CTSA,20-45892776-GA-G,2.0,Filtered,10,"[0.2, 0.8]",no*,"10_[0.2, 0.8]",0.843882
4,UDN004009,CTSA,20-45892776-GA-G,2.0,Filtered,10,"[0.25, 0.75]",no*,"10_[0.25, 0.75]",1.054852
...,...,...,...,...,...,...,...,...,...,...
1495,UDN970538,TMEM107,17-8173458-G-C,NaN,Filtered,50,"[0.25, 0.75]",no*,"50_[0.25, 0.75]",1.476793
1496,UDN970538,TMEM107,17-8173458-G-C,NaN,Filtered,60,"[0.1, 0.9]",no*,"60_[0.1, 0.9]",0.843882
1497,UDN970538,TMEM107,17-8173458-G-C,NaN,Filtered,60,"[0.15, 0.85]",no*,"60_[0.15, 0.85]",0.843882
1498,UDN970538,TMEM107,17-8173458-G-C,NaN,Filtered,60,"[0.2, 0.8]",no*,"60_[0.2, 0.8]",1.265823


In [27]:
import numpy as np
df = pd.DataFrame(data, columns=['GQ', 'VAF_range', 'Median_Rank','Mean_Rank', 'Variants_Lost'])
df['Percent_Lost'] = (df['Variants_Lost']/(509-35)) * 100

#df['Percent_Lost'] = ((df['Variants_Lost']-7)/(296-7)) * 100
df['Filtering'] =  df['GQ'] + '_' + df['VAF_range'].astype(str) 
df['VAF_range'] = df['VAF_range'].astype(str)
df = df.sort_values(by=['Median_Rank','Percent_Lost', 'GQ'],ascending=[False, True, True])

#df.to_csv('all_cohorts_df.tsv', sep='\t', index=None)
df

,GQ,VAF_range,Median_Rank,Mean_Rank,Variants_Lost,Percent_Lost,Filtering
0,OG,OG,6.0,43.531646,0,0.000000,OG_OG
1,10,"[0.1, 0.9]",3.5,27.542373,2,0.421941,"10_[0.1, 0.9]"
5,20,"[0.1, 0.9]",3.5,26.103814,2,0.421941,"20_[0.1, 0.9]"
2,10,"[0.15, 0.85]",3.0,23.019068,2,0.421941,"10_[0.15, 0.85]"
6,20,"[0.15, 0.85]",3.0,22.086864,2,0.421941,"20_[0.15, 0.85]"
9,30,"[0.1, 0.9]",3.0,24.815287,3,0.632911,"30_[0.1, 0.9]"
13,40,"[0.1, 0.9]",3.0,23.542553,4,0.843882,"40_[0.1, 0.9]"
17,50,"[0.1, 0.9]",3.0,22.546809,4,0.843882,"50_[0.1, 0.9]"
21,60,"[0.1, 0.9]",3.0,21.595745,4,0.843882,"60_[0.1, 0.9]"
10,30,"[0.15, 0.85]",2.0,21.184713,3,0.632911,"30_[0.15, 0.85]"


In [28]:
df=df[df['GQ']!='OG']

In [29]:
med_ranks = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(filled=True, size=50)).encode(
    x=alt.X('GQ'),
    y=alt.Y('Mean_Rank', scale=alt.Scale(domain=[15,30])),
    color=alt.Color('VAF_range:N', scale=alt.Scale(scheme='category20')),
   # shape=alt.Shape('VAF_range',scale=alt.Scale(range=['triangle-down','square','circle', 'triangle', 'diamond']) ),
    tooltip=['Percent_Lost', 'Median_Rank', 'VAF_range', 'Mean_Rank']
).properties(width=300)

percent_lost = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(filled=True, size=50)).encode(
    x=alt.X('GQ'),
    y=alt.Y('Percent_Lost'),
    color=alt.Color('VAF_range:N', scale=alt.Scale(scheme='category20')),
   # shape=alt.Shape('VAF_range',scale=alt.Scale(range=['triangle-down','square','circle', 'triangle', 'diamond']) ),
    tooltip=['Percent_Lost', 'Median_Rank', 'VAF_range', 'Mean_Rank']
).properties(width=300)

percent_lost
#hits=alt.Chart(str(graph), title='False Discovery Rate (FDR) Curve - By Gene ').mark_line(point=alt.OverlayMarkDef(filled=False, fill="white")).encode(


alt.hconcat(percent_lost, med_ranks).configure_legend(labelLimit=0, labelFontSize=15, titleFontSize=15, labelFont='arial').configure_axis(
    labelFontSize=13,
    titleFontSize=15,labelFont='arial'
    ).configure_scale(
    bandPaddingInner=0.9
    )

alt.HConcatChart(...)